In [916]:
import cPickle as pkl
from cspython.scraper import modifiedSoup
from cspython.data_processing import process_scrapped
import cPickle as pkl
import sys
sys.setrecursionlimit(15000)

import cspython.analysis as a
import scipy
print('scipy: {}'.format(scipy.__version__)) # numpy
import numpy as np
print('numpy: {}'.format(np.__version__))
# matplotlib


import matplotlib 
import matplotlib.pyplot as plt
print('matplotlib: {}'.format(matplotlib.__version__)) # pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
#import sklearn
#print('sklearn: {}'.format(sklearn.__version__))
#import xlrd

import seaborn as sns
sns.set_style("whitegrid")
%matplotlib inline


import sklearn
print('sklearn: {}'.format(sklearn.__version__))
#import xlrd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso, Ridge  
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import RFE
#import xgboost as xgb
from sklearn import cross_validation
from sklearn.model_selection import GridSearchCV
from sklearn import cross_validation #might be model_selection <--- this is the new one
from sklearn.preprocessing import Normalizer
import seaborn as sns
from sklearn import preprocessing 
sns.set_style("whitegrid")
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

scipy: 0.19.0
numpy: 1.11.3
matplotlib: 2.1.0
pandas: 0.22.0
sklearn: 0.18.1


In [917]:
with open('../cspython/test.pkl', 'rb') as f: 
     d = pkl.load(f)

In [918]:
big_data = process_scrapped(d)
overview, big_data = big_data

In [923]:
def combine_dfs(big_data):
    for l in range(len(big_data.keys())):
        series_data = big_data[big_data.keys()[l]]
        series_data_m = merge_matches(series_data)
        series_data_mo = merge_overview(series_data_m, series_data)
        series_data_mos = merge_scoreboards(series_data_mo, series_data)
        series_data_mosm = match_data_board_changer(series_data_mos, series_data)
        if l == 0:
            data = series_data_mosm
        else:
            data = pd.concat([data, series_data_mosm])  # THIS ALPHABATIZES THE COLUMNS! WE DONT WANT THAT IF POSSIBLE
    return data

In [915]:
def match_data_board_changer(series_data_mos,series_data):
    
    board_name = ['first_kills','who_kill_who', 'awp_kills']

    for idx, a in enumerate(series_data['match_data']):
        new_df = pd.DataFrame()
        for idx1, c in enumerate(board_name):    
            new_board_c = pd.DataFrame()
            new_board_r= pd.DataFrame()
            names_c = a[c].set_index('Unnamed: 0').columns
            for b in names_c:
                new_board_c[b+'_'+c] = a[c].set_index('Unnamed: 0')[b].str.split(pat = ':', expand = True)[0]
               
            names_r = a[c].set_index('Unnamed: 0').T.columns
            for b in names_r:
                new_board_r[b+'_'+c] = a[c].set_index('Unnamed: 0').T[b].str.split(pat = ':', expand = True)[1]
            new_board_c['nicknames'] = new_board_c.index
            new_board_r['nicknames'] = new_board_r.index
            
            board_df = new_board_c.append(new_board_r)
            
            if idx1 == 0:
                new_df = board_df
            else:
                new_df = pd.concat([new_df, board_df], axis = 1)
        if idx == 0:
            new_df['match_num'] = 1+idx
            con_df = new_df
        else:
            new_df['match_num'] = 1+idx
            con_df = con_df.append(new_df, ignore_index=True)
    con_df = con_df.loc[:, ~con_df.columns.duplicated()]
    series_data_mosm = pd.merge(series_data_mos, con_df, on=['nicknames', 'match_num'], how = 'inner')
    return series_data_mosm


In [886]:
def merge_matches(series_data):
    for d in range(0,len(series_data['matches'])):
        if d == 0:
            series_data['matches'][d] = series_data['matches'][d].rename(index = str, columns={ series_data['matches'][d].columns[10] : "team_A", series_data['matches'][d].columns[11] : "team_B" })
            series_data_m = series_data['matches'][d]
        else:
            series_data['matches'][d] = series_data['matches'][d].rename(index = str, columns={ series_data['matches'][d].columns[10] : "team_A", series_data['matches'][d].columns[11] : "team_B" })
            series_data_m = pd.concat([series_data_m, series_data['matches'][d]])
    return series_data_m
    #should work for concact the matches together


In [887]:
def merge_overview(series_data_m, series_data):
    series_data['match_overview'] = series_data['match_overview'].rename(index = str, columns ={series_data['match_overview'].columns[4]: "team_A_score",series_data['match_overview'].columns[5]: "team_B_score"})
    series_data_mo = pd.merge(series_data_m, series_data['match_overview'], on=['match_id', 'winner', 'map', 'series_id'])
    return series_data_mo
#works at merging matches with match_overviewb

In [888]:
def merge_scoreboards(series_data_mo, series_data):    
    for i in range(len(series_data['scoreboards'][0])):
        series_data['scoreboards'][0][i]['match_num'] = i+1
        series_data['scoreboards'][0][i] = series_data['scoreboards'][0][i].rename(index = str, columns={ series_data['scoreboards'][0][i].columns[0] : "team_players" })
        series_data['scoreboards'][1][i]['match_num'] = i+1
        series_data['scoreboards'][1][i] = series_data['scoreboards'][1][i].rename(index = str, columns={ series_data['scoreboards'][1][i].columns[0] : "team_players"})
        new_df = pd.concat([series_data['scoreboards'][0][i], series_data['scoreboards'][1][i]])
        
        if i == 0:
            con_df = new_df
        else:
            con_df = pd.concat([con_df, new_df])
   
    series_data_mos = pd.merge(series_data_mo, con_df, how='outer', on='match_num')
    series_data_mos['nicknames'] = series_data_mos['team_players'].str.split(expand = True)[3]
    return series_data_mos

       
        
#works at adding match_num to scoreboards
        
                         

In [919]:
data = combine_dfs(big_data)

In [921]:
for a in data.columns:
    print a

+/-
ADR
ANDROID-X23_awp_kills
ANDROID-X23_first_kills
ANDROID-X23_who_kill_who
BlackEagle_awp_kills
BlackEagle_first_kills
BlackEagle_who_kill_who
Blue +_awp_kills
Blue +_first_kills
Blue +_who_kill_who
CT
Edward_awp_kills
Edward_first_kills
Edward_who_kill_who
Empera_awp_kills
Empera_first_kills
Empera_who_kill_who
FNS_awp_kills
FNS_first_kills
FNS_who_kill_who
Fable_awp_kills
Fable_first_kills
Fable_who_kill_who
FalleN_awp_kills
FalleN_first_kills
FalleN_who_kill_who
Fog_awp_kills
Fog_first_kills
Fog_who_kill_who
Golden_awp_kills
Golden_first_kills
Golden_who_kill_who
GuardiaN_awp_kills
GuardiaN_first_kills
GuardiaN_who_kill_who
Hiko_awp_kills
Hiko_first_kills
Hiko_who_kill_who
JW_awp_kills
JW_first_kills
JW_who_kill_who
Jada_awp_kills
Jada_first_kills
Jada_who_kill_who
JustnatioN_awp_kills
JustnatioN_first_kills
JustnatioN_who_kill_who
K-D
KAST
KRIMZ_awp_kills
KRIMZ_first_kills
KRIMZ_who_kill_who
Kheops_awp_kills
Kheops_first_kills
Kheops_who_kill_who
Lekr0_awp_kills
Lekr0_first_kil

In [922]:
data.shape

(2831, 247)

In [400]:
data

,map,round_num,half,match_id,series_id,ending,CT,T,side_winner,winner,team_A,team_B,team_A_score,team_B_score,match_num,team_players,K-D,+/-,ADR,KAST,Rating2.0,nicknames
0,Mirage,7,1,9e6ab267-e3c7-4bf1-9e3b-f878b7de60be,01588838-693c-4098-8136-32690bd2045a,t_win,5POWER,Red Wolf,CT,5POWER,6,1,11,16,1,JiaMing 'originalheart' Gu originalheart,23-19,4,94.2,59.3%,1.26,originalheart
1,Mirage,7,1,9e6ab267-e3c7-4bf1-9e3b-f878b7de60be,01588838-693c-4098-8136-32690bd2045a,t_win,5POWER,Red Wolf,CT,5POWER,6,1,11,16,1,Daryl 'insanity' Chua insanity,21-22,-1,96.1,70.4%,1.14,insanity
2,Mirage,7,1,9e6ab267-e3c7-4bf1-9e3b-f878b7de60be,01588838-693c-4098-8136-32690bd2045a,t_win,5POWER,Red Wolf,CT,5POWER,6,1,11,16,1,Kevin 'dispenser' Te dispenser,20-20,0,75.5,74.1%,1.08,dispenser
3,Mirage,7,1,9e6ab267-e3c7-4bf1-9e3b-f878b7de60be,01588838-693c-4098-8136-32690bd2045a,t_win,5POWER,Red Wolf,CT,5POWER,6,1,11,16,1,Junlin 'Lieu' Luo Lieu,13-20,-7,63.9,70.4%,0.83,Lieu
4,Mirage,7,1,9e6ab267-e3c7-4bf1-9e3b-f878b7de60be,01588838-693c-4098-8136-32690bd2045a,t_win,5POWER,Red Wolf,CT,5POWER,6,1,11,16,1,Huang 'Mumu' Qiyu Mumu,13-22,-9,36.9,55.6%,0.61,Mumu
5,Mirage,7,1,9e6ab267-e3c7-4bf1-9e3b-f878b7de60be,01588838-693c-4098-8136-32690bd2045a,t_win,5POWER,Red Wolf,CT,5POWER,6,1,11,16,1,Kitikawin 'PTC' Rattanasukol PTC,24-12,12,88.5,74.1%,1.55,PTC
6,Mirage,7,1,9e6ab267-e3c7-4bf1-9e3b-f878b7de60be,01588838-693c-4098-8136-32690bd2045a,t_win,5POWER,Red Wolf,CT,5POWER,6,1,11,16,1,Batbayar 'kabal' Bat-Enkh kabal,28-18,10,101.7,74.1%,1.50,kabal
7,Mirage,7,1,9e6ab267-e3c7-4bf1-9e3b-f878b7de60be,01588838-693c-4098-8136-32690bd2045a,t_win,5POWER,Red Wolf,CT,5POWER,6,1,11,16,1,Peng 'xiaosaGe' Song xiaosaGe,20-20,0,79.8,63.0%,1.03,xiaosaGe
8,Mirage,7,1,9e6ab267-e3c7-4bf1-9e3b-f878b7de60be,01588838-693c-4098-8136-32690bd2045a,t_win,5POWER,Red Wolf,CT,5POWER,6,1,11,16,1,Gan-Erdene 'dobu' Batbold dobu,15-17,-2,66.3,77.8%,0.96,dobu
9,Mirage,7,1,9e6ab267-e3c7-4bf1-9e3b-f878b7de60be,01588838-693c-4098-8136-32690bd2045a,t_win,5POWER,Red Wolf,CT,5POWER,6,1,11,16,1,Lei 'forget' Li forget,16-23,-7,71.1,66.7%,0.78,forget


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data

In [ ]:
data = pd.concat([data,data['K-D'].str.split('-', 1, expand=True).rename(columns={0:'player_kill', 1:'player_death'})], axis=1)


In [ ]:
data.shape

In [ ]:
data = data.apply(pd.to_numeric, errors='ignore')

In [ ]:
data[['map', 'ending', 'CT', 'T', 'side_winner','winner', 'team_players']]= data[['map', 'ending', 'CT', 'T', 'side_winner','winner', 'team_players']].apply(lambda x: pd.factorize(x)[0]) 

In [ ]:
data['KAST'] = data['KAST'].apply(lambda x : float(x.strip('%')) / 100.0)

In [ ]:
data.dtypes

In [ ]:
corr = data.select_dtypes(include = ['float64', 'int64', 'int32']).iloc[:, 0:].corr()
plt.figure(figsize=(25, 25))
sns.heatmap(corr, vmax=1, square=True)



In [ ]:
ydata = data['winner']

In [ ]:
data = data.drop('winner', 1)

In [ ]:
data = data.drop('match_id', 1)

In [ ]:
data = data.drop('series_id', 1)

In [ ]:
data = data.drop('K-D', 1)

In [ ]:
data.shape

In [ ]:
y = ydata.values
X = data.values[:,0:19]

In [ ]:
num_folds = 10
num_instances = len(X) 
seed = 7
scoring = 'accuracy'

In [ ]:
def local_cv(model, params):                             #KFOLD WITH GRID SEARCH
    param_grid = params
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring=scoring, cv=kfold)
    grid_result = grid.fit(X, y)
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    for params, mean_score, scores in grid_result.grid_scores_:
        print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

In [ ]:
num_folds = 10
num_instances = len(X) 
seed = 7
scoring = 'accuracy'
model = GradientBoostingClassifier()
params = {
 'max_depth':[4,6,8,10,12,16],
 'n_estimators':[20,30,50,80,100],
'learning_rate':[.1,.01,.001]}
local_cv(model, params)

In [ ]:
models = []
#models.append(('LR', LogisticRegression()))
#models.append(('LASSO', Lasso())) 
#models.append(('Ridge', Ridge())) 
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('NB', GaussianNB()))
#models.append(('CART', DecisionTreeClassifier())) 
#models.append(('KNeighborsClassifier', KNeighborsClassifier())) 
#models.append(('XGBClassifier', xgb.XGBClassifier()))
models.append(('GradientBoostingClassifier', GradientBoostingClassifier()))
#models.append(('AdaBoostClassifier', AdaBoostClassifier()))
#models.append(('RandomForestClassifier', RandomForestClassifier()))
#models.append(('ExtraTreesClassifier', ExtraTreesClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('SVC', SVC()))

# evaluate each model in turn
results = []
names = []
for name, model in models:
    print '1'
    kfold = cross_validation.KFold(n=num_instances, n_folds=num_folds, random_state=seed)
    print '2'
    cv_results = cross_validation.cross_val_score(model, X, y, cv=kfold, scoring = scoring)
    print '3'
    results.append(cv_results)
    print '4'
    names.append(name)
    print '5'
    model.fit(X, y)
    print '6'
    predicted = model.predict(X)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)